In [1]:
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis,spinless_fermion_basis_1d,boson_basis_1d, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [11]:
L = 5
M = 0.6
l=1
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [12]:
basis_spin =spin_basis_1d(L=L,S = str(l))
basis_fermions = spinless_fermion_basis_1d(L=L+1,Nf = [0,1,2,3,4])
basis=tensor_basis(basis_spin,basis_fermions)
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
	    0.         |2 2 2 2 2>           242         |1 1 1 1 0 0>           60  
	    1.         |2 2 2 2 2>           242         |1 1 1 0 1 0>           58  
	    2.         |2 2 2 2 2>           242         |1 1 1 0 0 1>           57  
	    3.         |2 2 2 2 2>           242         |1 1 1 0 0 0>           56  
	    4.         |2 2 2 2 2>           242         |1 1 0 1 1 0>           54  
	    5.         |2 2 2 2 2>           242         |1 1 0 1 0 1>           53  
	    6.         |2 2 2 2 2>           242         |1 1 0 1 0 0>           52  
	    7.         |2 2 2 2 2>           242         |1 1 0 0 1 1>           51  
	    8.         |2 2 2 2 2>           242         |1 1 0 0 1 0>           50  
	    9.         |2 2 2 2 2>           242         |1 1 0 0 0 1>           49  
	   10.         |2 2 2 2 2>           242         |1 1 0 0 0 0>           48  
	   11.         |2 2 2 2 2>           242         |1 0 1 1 1 0

In [13]:
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [14]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z|",liste[0]],
            ["|n",liste_m[0]],
        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z|",liste_m[i-1]],
            ["|n",liste_m[i]]
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z|",liste_m[i-1]],
            ["I|",liste[i-1]],
            ["|n",liste_m[i]],
        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z|",liste[i]],
            ["z|",liste_m[i-1]],
            ["|n",liste_m[i]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z|",liste[i]],
            ["z|",liste_m[i-1]],  
            ["I|",liste[i]],
            ["|n",liste_m[i]],
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))
G = 0
for i in range(L+1):
    G += Gauss_law[i]**2
for i in range(basis.Ns):
    if (G.expt_value(basis_vectors[basis.Ns-i-1])<=0.+1e-8) & (G.expt_value(basis_vectors[basis.Ns-i-1])>=0.-1e-8):
        print(i)

6
179
237
1552
1725
1783
2242
2300
2473
6184
6357
6415
6874
6932
7105
8421
8479
8652
9169


In [15]:
kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[M*(-1)**i,i] for i in range(L+1)]
interaction_term = [[-1j/(2*np.sqrt(2)),i,i,i+1] for i in range(L)]

hamiltonian_map = [
    
    ["zz|",kin_energy],
    ["|n",mass_term],
    ["|I",mass_term],

    ["+|+-",interaction_term],
    ["-|-+",interaction_term]
]



Ham_LGT = hamiltonian(hamiltonian_map,dynamic_list = [],basis=basis,**no_checks)
Hamiltonian_LGT = Ham_LGT + 1000*G
ham_matrix = Hamiltonian_LGT.toarray()

In [16]:
def check_hermitian(a, rtol=1e-10, atol=1e-10):
    return np.allclose(a, a.conj().T, rtol=rtol, atol=atol)

In [17]:
print(check_hermitian(ham_matrix))

True


In [18]:
eigenval,eigenvec = np.linalg.eigh(ham_matrix)

In [19]:
print(eigenval[eigenval<=1e2])

[-2.42109438 -0.50953275 -0.43195924 -0.30195291 -0.16060728 -0.10869319
  0.58433579  0.58693727  0.7261395   1.45489394  1.59198359  1.76802289
  1.85978551  1.9358955   1.98573557  2.16777235  2.75447228  2.82307748
  4.0947881 ]
